In [ ]:
#Imports Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
from fiona import BytesCollection 

## 1. WFS abrufen

In [ ]:
url = 'https://services.geo.sg.ch/wss/service/SG00164_WFS/guest?request=DescribeFeatureType&service=WFS&REQUEST=GetFeature&TYPENAME=SG00164:Laermemission'

In [ ]:
driver = webdriver.Firefox()

In [ ]:
driver.get(url)

In [ ]:
page = driver.page_source.encode('utf-8')

In [ ]:
page

## 2. Informationen aus dem Infowust auslesen (BytesCollegtion)

In [ ]:
with BytesCollection(page) as f:
        gdf = gpd.GeoDataFrame.from_features(f)
       
    

In [ ]:
gdf

In [ ]:
gdf.plot()

In [ ]:
gdf.shape

In [ ]:
gdf.dtypes

In [ ]:
gdf.to_csv(r'daten/larmemission-street.csv', index = False)

In [ ]:
df = pd.read_csv('daten/larmemission-street.csv')

In [ ]:
df[(gdf['Emissionswert__Lre__Tag__dB_A__'] == 75) & (gdf['Emissionswert__Lre__Nacht__dB_A__'] == 63)]

## 3. Einen Geopunkt aus dem Multilinestring herausrechnen
Um später auf Grund dessen die Adresse bestimmen zu können

In [ ]:
df['gml_id'].loc[0]

In [ ]:
df['geometry'].iloc[0]

In [ ]:
# Split Koordinaten in geometry beim Komma und packt das Resultat in eine Liste
geopoint = []

for i in range(len(df)):
    elem = df['geometry'].str.split(",")[i]

  
    geopoint.append(elem)

   

In [ ]:
#Die Liste in ein df transformieren 
df_point = pd.DataFrame(geopoint)

In [ ]:
df_point

In [ ]:
# Das df mit den Daten und jenes mit den Geopoints (aber nur Spalte 1) auf dem Index mergen
df_new = pd.merge(df,df_point[[1]], left_index=True, right_index=True)

In [ ]:
df_new.tail()

In [ ]:
df_new.rename(columns={1: 'Coordinates'}, inplace=True)

## 4. Die beiden Koordinaten aufsplitten und in einzelne Spalten packen
(Lat = north, Long = east)

In [ ]:
# Koordinaten in column Geopoints bei den Leerstellen splitten (dabei geht das Null verloren, das beinflusst aber die Koordinaten nicht bemerkenswert)
coordinates = []

for i in range(len(df_new)):
    elem = df_new['Coordinates'].str.split(' ')[i]
    coordinates.append(elem)

In [ ]:
coordinates

In [ ]:
coordinates[10][1]

In [ ]:
# aus der Liste die Koordinaten auslesen 

long = []
lat = []

for i in range(len(coordinates)):
    elem_lo = coordinates[i][1]
    long.append(elem_lo)
    
    elem_lat = coordinates[i][2]
    lat.append(elem_lat)
    

In [ ]:
lat

In [ ]:
d = {'longitude':long, 'latitude':lat}

In [ ]:
df_coordinates = pd.DataFrame(d)

In [ ]:
df_coordinates

In [ ]:
df_final = pd.merge(df_new,df_coordinates, left_index=True, right_index=True)

In [ ]:
df_final

In [ ]:
df_final[df_final['Emissionswert__Lre__Tag__dB_A__'] == 83]

## 5. Df vor dem Export etwas bereinigen

In [ ]:
df_final.rename({'Emissionswert__Lre__Tag__dB_A__' : 'Emissionswert Tag',
                 'Emissionswert__Lre__Nacht__dB_A__':'Emissionswert Nacht',
                 'Durchschnittlicher_Tagesverkehr__DTV___Fz_d_' : 'Tagesverkehr Jahresmittel',
'Anzahl_Fahrzeuge_Tag__Nt___Fz_h_': 'Anzahl Fz Tag pro h',
'Anteil_lauter_Fahrzeuge_Tag__Nt2_____':'Prozentanteil lauter Fz Tag',
'Anzahl lauter Fahrzeuge Tag (Fz/h)':'Anzahl lauter Fz Tag pro h',
'Anzahl_Fahrzeuge_Nacht__Nn___Fz_h_':'Anzahl Fz Nacht pro h',
'Anteil_lauter_Fahrzeuge_Nacht__Nn2_____':'Prozentanteil lauter Fz Nacht',
'Anzahl lauter Fahrzeuge Nacht (Fz/h)':'Anzahl lauter Fz Nacht pro h',
                'Steigung____' : 'Steigung',
                 'Belagskorrektur__dB_A__' : 'Belagskorrektur',
'Signalisierte_Geschwindigkeit_Tag__km_h_':'Tempo'},
                
                
                inplace=True, axis = 1)

### Alle rows mit Wert 0 in beiden Angaben zur Emission entfernen 

In [ ]:
#In den beiden Spalten 0 durch NaN ersetzen
cols = ["Emissionswert Tag","Emissionswert Nacht"]
df_final[cols] = df_final[cols].replace({'0':np.nan, 0:np.nan})

#rows mit NaN in diesen beiden Spalten entfernen
df_final.dropna(subset=["Emissionswert Tag","Emissionswert Nacht"], inplace = True)

In [ ]:
df_final.to_csv(r'daten/larmemission-streetFULL.csv', index = False)
